In [25]:
import requests
import pandas as pd
import json
import os
import pyarrow as pa
import pyarrow.json as pjson
import polars as pl

In [11]:
if not os.path.exists("./stazioni_meteo.json"):
    content = requests.get(
        "https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/"
    ).content
    stazioni = json.loads(content)
    with open("stazioni_meteo.json", "wb") as f:
        f.write(content)
else:
    with open("stazioni_meteo.json", "rt") as f:
        stazioni = json.load(f)

In [24]:
pd.DataFrame.from_records(stazioni["results"]).query("")

0      https://utility.arpa.piemonte.it/meteoidro/pun...
1      https://utility.arpa.piemonte.it/meteoidro/pun...
2      https://utility.arpa.piemonte.it/meteoidro/pun...
3      https://utility.arpa.piemonte.it/meteoidro/pun...
4      https://utility.arpa.piemonte.it/meteoidro/pun...
                             ...                        
331    https://utility.arpa.piemonte.it/meteoidro/pun...
332    https://utility.arpa.piemonte.it/meteoidro/pun...
333    https://utility.arpa.piemonte.it/meteoidro/pun...
334    https://utility.arpa.piemonte.it/meteoidro/pun...
335    https://utility.arpa.piemonte.it/meteoidro/pun...
Name: fk_id_punto_misura_meteo, Length: 336, dtype: object

In [31]:
data = pl.DataFrame(
    stazioni["results"],
    schema_overrides={
        "url": pl.Utf8(),
        "codice_istat_comune": pl.Utf8(),
        "sensori_meteo": pl.List(
            pl.Struct(
                {
                    "url": pl.Utf8(),
                    "id_parametro": pl.Utf8(),
                    "data_inizio": pl.Utf8(),
                    "data_fine": pl.Utf8(),
                    "quota_da_pc": pl.Float32(),
                    "altezza_supporto": pl.Float32(),
                    "note": pl.Utf8(),
                    "fk_id_stazione_meteorologica": pl.Utf8(),
                }
            )
        ),
    },
)

In [40]:
thermo_sensors = (
    data.rename({"url": "url_stazione"})
    .explode("sensori_meteo")
    .with_columns(
        pl.col("sensori_meteo").struct.rename_fields(
            [
                "url_sensore",
                "id_parametro",
                "data_inizio_sensore",
                "data_fine_sensore",
                "quota_da_pc",
                "altezza_supporto",
                "note_sensore",
                "copy_url_stazione",
            ]
        )
    )
    .unnest("sensori_meteo")
    .filter(
        pl.col("id_parametro") == "TERMA"
    )
)

In [41]:
thermo_sensors.write_csv("thermo_sensors.csv", quote_style="always")

In [48]:
urls = thermo_sensors.get_column("fk_id_punto_misura_meteo").to_list()

In [49]:
urls

['https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001003-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001010-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001011-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001013-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001019-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001019-901/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001019-901/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001022-900/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001022-901/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001022-902/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001022-903/',
 'https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/PIE-001022-905/',
 'https://utilit